In [1]:
# load the json file test_queries.json
import json

test_queries = json.load(open('test_queries.json'))

In [3]:
'''
test_queries looks like this:

[
    {
      "task": "renewable_energy_presentation",
      "prompt": "Create a presentation about the benefits of renewable energy. Include a pie chart showing the global energy mix and a bar plot comparing the costs of different energy sources. Use a green technology theme and make it look professional."
    },
    {
      "task": "ai_healthcare_research_summary",
      "prompt": "Make a presentation summarizing the key findings of the research paper 'Advances in AI for Healthcare' by John Doe et al. Include the most important figures and tables from the paper. Use a clean, minimalistic theme suitable for an academic audience."
    },
    ...
'''
# print queries whose task is 'renewable_energy_presentation', 'product_sales_data_formatting', 'software_engineer_cover_letter'
filtered_queries = [query for query in test_queries if query['task'] in ['renewable_energy_presentation', 'product_sales_data_formatting', 'software_engineer_cover_letter']]
print(json.dumps(filtered_queries, indent=2))

[
  {
    "task": "renewable_energy_presentation",
    "prompt": "Create a presentation about the benefits of renewable energy. Include a pie chart showing the global energy mix and a bar plot comparing the costs of different energy sources. Use a green technology theme and make it look professional."
  },
  {
    "task": "product_sales_data_formatting",
    "prompt": "Create an Excel sheet with a table of sales data for different products. Include columns for product name, unit price, quantity sold, and total revenue. Calculate the average price and total revenue for each product. Format the column names in bold and merge the cells in the header row."
  },
  {
    "task": "software_engineer_cover_letter",
    "prompt": "Draft a professional cover letter for a software engineer position at Google. Highlight the candidate's experience with Python, machine learning, and web development. Use a formal tone and a clean, modern template."
  }
]


In [4]:
'''
Rough notes:
UFO paper - https://arxiv.org/abs/2402.07939
OS Copilot paper - https://arxiv.org/pdf/2402.07456
ScreenAgent paper - https://arxiv.org/abs/2402.07945
'''

'\nRough notes:\nUFO paper - https://arxiv.org/abs/2402.07939\n'

In [39]:
import os
import re

def prompt_loader(agent_name):
    file_name = f"../api/prompt_lib/{agent_name}.txt"
    
    if not os.path.exists(file_name):
        return None
    
    with open(file_name, 'r') as file:
        content = file.read()
    
    prompt_template = {
        'system_prompt': '',
        'fewshot_messages': []
    }
    
    system_prompt_pattern = r'\[\[prompt_template-system_prompt\]\](.*?)\[\[prompt_template-system_prompt\]\]'
    system_prompt_match = re.search(system_prompt_pattern, content, re.DOTALL)
    
    if system_prompt_match:
        prompt_template['system_prompt'] = system_prompt_match.group(1).replace('\\"', '"').strip()
    
    fewshot_pattern = r'\[\[prompt_template-fewshot\]\](.*?)\[\[prompt_template-fewshot\]\]'
    fewshot_match = re.search(fewshot_pattern, content, re.DOTALL)
    
    if fewshot_match:
        fewshot_content = fewshot_match.group(1).replace('\\"', '"').strip()
        fewshot_messages = re.split(r'\[\[prompt_template-fewshot-(?:user|assistant)\]\]', fewshot_content)
        
        for message in fewshot_messages:
            message = message.strip()
            if message:
                if message.startswith('{') and message.endswith('}'):
                    prompt_template['fewshot_messages'].append({'role': 'assistant', 'content': message})
                else:
                    prompt_template['fewshot_messages'].append({'role': 'user', 'content': message})
    
    return prompt_template
task_corrector_prompts = prompt_loader('task_corrector')

In [40]:
prompt_templates = prompt_loader("task_corrector")
prompt_templates['fewshot_messages']

[{'role': 'user',
  'content': 'Correct the following task\n<task>\nWrite a press release announcing the launch of a new sustainable fashion line. Include quotes from the designer and information about the eco-friendly materials used. Format the document with a header, subheadings, and a footer with contact information.\n</task>\n\nNew App list:\n<app_list>\n{\n    "AllUsersPrograms": {\n      "Access.lnk": null,\n      "Brave.lnk": null,\n    },\n    "StartMenu": {\n      "Programs": {\n        "Administrative Tools.lnk": null,\n        "F1_23.lnk": null,\n        "File Explorer.lnk": null\n      }\n    },\n    "AllUsersDesktop": {\n      "Brave.lnk": null,\n      "VLC media player.lnk": null\n    }\n  }\n</app_list>'},
 {'role': 'assistant',
  'content': '{"corrected_task": "Write a press release using Google Docs announcing the launch of a new sustainable fashion line. Use Brave to google information about a new sustainable fashion line and include quotes from the designer and infor

In [24]:
import re

def xmlResponseToDict(xml_response):
    def parse_tag(tag_content):
        if not tag_content:
            return None
        
        tag_pattern = re.compile(r'<(\w+)>(.*?)</\1>', re.DOTALL)
        tag_match = tag_pattern.match(tag_content)
        
        if tag_match:
            tag_name = tag_match.group(1)
            tag_value = tag_match.group(2).strip()
            
            if tag_name.endswith('_list'):
                items = re.findall(r'<(\w+)>(.*?)</\1>', tag_value, re.DOTALL)
                
                if items:
                    parsed_items = []
                    for item_name, item_content in items:
                        parsed_item = parse_tag(f'<{item_name}>{item_content}</{item_name}>')
                        if parsed_item:
                            parsed_items.append(parsed_item)
                    
                    return {tag_name: parsed_items}
                else:
                    items = re.split(r'\s*\n\s*', tag_value)
                    items = [item.strip() for item in items if item.strip()]
                    return {tag_name: items}
            else:
                return {tag_name: tag_value}
        
        return None

    tags = re.findall(r'<(\w+)>(.*?)</\1>', xml_response, re.DOTALL)
    result = {}
    
    for tag_name, tag_content in tags:
        parsed_tag = parse_tag(f'<{tag_name}>{tag_content}</{tag_name}>')
        if parsed_tag:
            result.update(parsed_tag)
    
    return result

In [25]:
xml_test_string = '''
<example_tag>
Multiline strings 1
Multiline strings 2
Multiline strings 3
Multiline strings 4
Multiline strings 5
</example_tag>

<example_tag_list>
list item 1
list item 2
list item 3
list item 4
</example_tag_list>

<example_tag1_list>

<example_tag1>
asdf asdf asdf
</example_tag1>

<example_tag2>
asdf asdf asdf
</example_tag2>

<example_tag_nested_list>
list item 1
list item 2
list item 3
list item 4
</example_tag_nested_list>

</example_tag1_list>
'''

In [26]:
dict_test = xmlResponseToDict(xml_test_string)
# pretty print the dict
import json
print(json.dumps(dict_test, indent=2))

{
  "example_tag": "Multiline strings 1\nMultiline strings 2\nMultiline strings 3\nMultiline strings 4\nMultiline strings 5",
  "example_tag_list": [
    "list item 1",
    "list item 2",
    "list item 3",
    "list item 4"
  ],
  "example_tag1_list": [
    {
      "example_tag1": "asdf asdf asdf"
    },
    {
      "example_tag2": "asdf asdf asdf"
    },
    {
      "example_tag_nested_list": [
        "list item 1",
        "list item 2",
        "list item 3",
        "list item 4"
      ]
    }
  ]
}


In [22]:
import re

def xmlResponseToDict(xml_response):
    def parse_tag(tag_content):
        tag_pattern = re.compile(r'<(\w+)>(.*?)</\1>', re.DOTALL)
        tag_match = tag_pattern.match(tag_content)
        
        if tag_match:
            tag_name, tag_value = tag_match.groups()
            
            if tag_name.endswith('_list'):
                items = re.findall(r'<(\w+)>(.*?)</\1>', tag_value, re.DOTALL)
                if items:
                    return [{item[0]: parse_tag(item[1])} for item in items]
                else:
                    return re.split(r'\n', tag_value.strip())
            
            return tag_value.strip()
        
        return None

    tags = re.findall(r'<(\w+)>(.*?)</\1>', xml_response, re.DOTALL)
    result = {}
    for tag in tags:
        parsed_tag = parse_tag(tag[1])
        if parsed_tag is not None:
            result[tag[0]] = parsed_tag
    return result